<a href="https://colab.research.google.com/github/oshmita26/Automatic-number-plate-recognition-with-PaddleOCR/blob/main/Fine_tune_ppocr_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print()

## PaddleOCR Installation

In [ ]:
!pip install paddlepaddle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.7/121.7 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.2/383.2 kB 37.4 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-api-core 2.11.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 3.20.0 which is incompatible.
google-cloud-bigquery 3.9.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, b

In [ ]:
!pip install "paddleocr>=2.0.6"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 445.9/445.9 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 813.9/813.9 kB 61.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 77.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 86.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 101.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
## We are going to retrain the ppocr model on our own custom dataset
!mkdir testing

In [ ]:
!mkdir testing/class

In [ ]:
!mkdir training

In [ ]:
!mkdir training/class

In [ ]:
import os
import random
import shutil

data_path = '/content/drive/MyDrive/final_aug_dataset'  # Path to your dataset
train_ratio = 0.8  # Ratio of training data (e.g., 0.8 means 80% for training and 20% for validation)

# Create directories for training and validation data
train_data_path = '/content/training/class'  # Path to store training data
valid_data_path = '/content/testing/class'  # Path to store validation data

os.makedirs(train_data_path, exist_ok=True)
os.makedirs(valid_data_path, exist_ok=True)

# Get a list of all images in the dataset directory
image_files = [file for file in os.listdir(data_path) if file.endswith('.jpeg')]

# Shuffle the image files randomly
random.shuffle(image_files)

# Calculate the number of images for training and validation
num_train = int(len(image_files) * train_ratio)
num_valid = len(image_files) - num_train

# Copy images to the training directory
for image_file in image_files[:num_train]:
    src_path = os.path.join(data_path, image_file)
    dst_path = os.path.join(train_data_path, image_file)
    shutil.copy(src_path, dst_path)

# Copy images to the validation directory
for image_file in image_files[num_train:]:
    src_path = os.path.join(data_path, image_file)
    dst_path = os.path.join(valid_data_path, image_file)
    shutil.copy(src_path, dst_path)

print(f"Train data: {num_train} images")
print(f"Validation data: {num_valid} images")

Train data: 884 images
Validation data: 222 images


In [ ]:
import paddle
import paddle.nn as nn
import paddle.optimizer as optim
from paddle.io import Dataset, DataLoader
from paddle.vision.transforms import transforms
from paddle.vision.datasets import DatasetFolder
from paddle.vision.models import resnet50

extensions = ['.jpg', '.png', '.jpeg']
# Step 1: Prepare your training data (annotated dataset)
train_data_path = '/content/training'  # Path to your training dataset
valid_data_path = '/content/testing'  # Path to your validation dataset
num_classes = 10  # Number of number plate classes

In [ ]:
# Step 2: Create a custom dataset class
class NumberPlateDataset(Dataset):
    def __init__(self, data_path):
        self.data_path = data_path
        self.transforms = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

        # Load images and annotations from your dataset
        self.dataset = DatasetFolder(data_path, transform=self.transforms, extensions=extensions)

    def __getitem__(self, index):
        image, label = self.dataset[index]
        return image, label

    def __len__(self):
        return len(self.dataset)

# Step 3: Define the model architecture
class NumberPlateRecognitionModel(nn.Layer):
    def __init__(self, num_classes):
        super(NumberPlateRecognitionModel, self).__init__()
        self.base_model = resnet50(pretrained=True)
        self.fc = nn.Linear(1000, num_classes)  # Modify the output features to num_classes

    def forward(self, x):
        x = self.base_model(x)
        x = paddle.flatten(x, start_axis=1)
        x = paddle.reshape(x, shape=[-1, 1000])   ## Reshaping the x-tensor to 1000 since that requires matching to k
        x = self.fc(x)
        return x

# Step 4: Set up the fine-tuning pipeline
def fine_tune():
    # Set up training parameters
    batch_size = 16
    num_epochs = 10
    learning_rate = 0.001

    # Create custom datasets for training and validation
    train_dataset = NumberPlateDataset(train_data_path)
    valid_dataset = NumberPlateDataset(valid_data_path)

    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

    # Create the model instance
    model = NumberPlateRecognitionModel(num_classes)

    # Set up the loss function
    criterion = nn.CrossEntropyLoss()

    # Set up the optimizer and learning rate scheduler
    optimizer = paddle.optimizer.Adam(learning_rate=learning_rate, parameters=model.parameters())
    scheduler = paddle.optimizer.lr.StepDecay(learning_rate=learning_rate, step_size=3, gamma=0.1)

    # Set up the training loop
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0

        for batch in train_loader:
            images, labels = batch

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.numpy().item()

            # Backward pass and optimization
            loss.backward()
            optimizer.step()
            optimizer.clear_grad()

        # Compute average loss for the epoch
        avg_loss = total_loss / len(train_loader)

        # Validate the model
        model.eval()
        total_correct = 0
        total_samples = 0

        for batch in valid_loader:
            images, labels = batch

            # Forward pass
            outputs = model(images)
            predicted = paddle.argmax(outputs, axis=1)

            # Compute accuracy
            total_correct += paddle.sum(predicted == labels).numpy().item()
            total_samples += labels.shape[0]

        accuracy = total_correct / total_samples

        # Update learning rate scheduler
        scheduler.step()

        # Print training progress
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")

# Step 5: Start the fine-tuning process
fine_tune()

Epoch [1/10], Loss: 0.0731, Accuracy: 1.0000
Epoch [2/10], Loss: 0.0000, Accuracy: 1.0000
Epoch [3/10], Loss: 0.0000, Accuracy: 1.0000
Epoch [4/10], Loss: 0.0000, Accuracy: 1.0000
Epoch [5/10], Loss: 0.0000, Accuracy: 1.0000
Epoch [6/10], Loss: 0.0000, Accuracy: 1.0000
Epoch [7/10], Loss: 0.0000, Accuracy: 1.0000
Epoch [8/10], Loss: 0.0000, Accuracy: 1.0000
Epoch [9/10], Loss: 0.0000, Accuracy: 1.0000
Epoch [10/10], Loss: 0.0000, Accuracy: 1.0000
